# Calibrate Trained FasterRisk on eICU dataset

In [ ]:
import mimic_pipeline.utils as utils
from mimic_pipeline.metric import get_calibration_curve, get_model_size
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import brier_score_loss
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

eicu_df = pd.read_csv('data/eICU-union.csv')
print(eicu_df.shape)
eicu_df.head()

In [ ]:
X_test, y_test = eicu_df.drop('hospital_expire_flag', axis=1), eicu_df['hospital_expire_flag']
X_test.shape

In [ ]:
X_rest, X_cali, y_rest, y_cali = train_test_split(X_test, y_test, test_size=2000, random_state=utils.SEED, stratify=y_test)     # use 2000 patients as calibration set
oasis_rest_prob, sapsii_rest_prob, apacheiv_rest_prob, apacheiva_rest_prob = X_rest['oasis_prob'], X_rest['sapsii_prob'], X_rest['apache_iv_prob'], X_rest['apache_iva_prob']
oasis_cali_prob, sapsii_cali_prob, apacheiv_cali_prob, apacheiva_cali_prob = X_cali['oasis_prob'], X_cali['sapsii_prob'], X_cali['apache_iv_prob'], X_cali['apache_iva_prob']

In [ ]:
print(X_rest.shape)
X_rest.head()

In [ ]:
print(X_cali.shape)
X_cali.head()

In [ ]:
rest = pd.concat([X_rest, y_rest], axis=1)
cali = pd.concat([X_cali, y_cali], axis=1)
rest.to_csv('data/eICU-union-noncali.csv', index=False)
cali.to_csv('data/eICU-union-cali.csv', index=False)

In [ ]:
X_rest = X_rest.drop(['uniquepid', 'patientunitstayid', 'apache_iv_prob', 'apache_iva_prob', 'oasis_prob', 'sapsii_prob'], axis=1)
X_cali = X_cali.drop(['uniquepid', 'patientunitstayid', 'apache_iv_prob', 'apache_iva_prob', 'oasis_prob', 'sapsii_prob'], axis=1)

In [ ]:

import joblib

def calibrate_fasterrisk(X_rest, y_rest, X_cali, y_cali, group_sparsity, plot=False, method='isotonic', save=False):
    assert  np.abs((len(y_cali[y_cali == 1]) / len(y_cali) )- ( len(y_rest[y_rest == 1]) / len(y_rest) )) < 1e-3
    
    fasterrisk_model = joblib.load(f"models/fasterrisk/fasterrisk-{group_sparsity}")
    binarizer = joblib.load(f"models/fasterrisk/fasterrisk-{group_sparsity}-binarizer")
    X_cali, _ = binarizer.transform(X_cali)
    y_prob_cali = fasterrisk_model.predict_proba(X_cali.to_numpy())
    print(f"y_prob_cali_min: {np.min(y_prob_cali)}, y_prob_cali_max: {np.max(y_prob_cali)}")
    X_rest, _ = binarizer.transform(X_rest)
    y_prob_rest = fasterrisk_model.predict_proba(X_rest.to_numpy())
    print(f"y_prob_rest_min: {np.min(y_prob_rest)}, y_prob_rest_max: {np.max(y_prob_rest)}")
    
    if method == 'isotonic':
        calibrator = IsotonicRegression(out_of_bounds='clip').fit(y_prob_cali, y_cali)
        print(f"Isotonic Regression Min: {calibrator.X_min_}, Max: {calibrator.X_max_}")
        y_prob_rest = calibrator.predict(y_prob_rest)
    elif method == 'sigmoid':
        calibrator = LogisticRegression(solver='lbfgs', penalty=None, random_state=utils.SEED).fit(y_prob_cali.reshape(-1, 1), y_cali)
        assert get_model_size(calibrator) == 2
        y_prob_rest = calibrator.predict_proba(y_prob_rest.reshape(-1, 1))[:, 1]
    
    if save:
        joblib.dump(calibrator, f"models/fasterrisk/fasterrisk-{group_sparsity}-calibrator")
        print(f"SAVE: Calibrator saved to models/fasterrisk/fasterrisk-{group_sparsity}-calibrator")
    assert np.min(y_prob_rest) >= 0 and np.max(y_prob_rest) <= 1
    _, _, h, p = get_calibration_curve(y_rest, y_prob_rest, strategy='quantile')
    prob_true, prob_pred, _, _ = get_calibration_curve(y_rest, y_prob_rest)
    
    if plot:
        sns.set_style("ticks")
        sns.lineplot(x=np.linspace(0,1), y=np.linspace(0,1), color='black', linestyle='--', linewidth=1)
        ax = sns.lineplot(x=prob_pred, y=prob_true, color='red', linewidth=1, marker='s', label=f"FasterRisk ({group_sparsity})")
        ax.figure.set_size_inches(7, 7)
        plt.xlabel("Predicted Probability")
        plt.ylabel("True Probability")
        plt.legend()
        plt.show()
    smr = np.sum(y_test.replace({-1: 0})) / np.sum(y_prob_rest)
    print(f"Fasterrisk {group_sparsity}\nBrier score: {brier_score_loss(y_rest, y_prob_rest):.3f}\nHosmer-Lemeshow C-stat: {h:.3f}, p-value: {p:.3f}\nSMR: {smr:.3f}\n")

In [ ]:
def calibrate_scores(score, y_rest, y_prob_rest, y_cali, y_prob_cali, plot=False, method='isotonic', save=False):
    assert  np.abs((len(y_cali[y_cali == 1]) / len(y_cali) )- ( len(y_rest[y_rest == 1]) / len(y_rest) )) < 1e-3
    
    if method == 'isotonic':
        calibrator = IsotonicRegression(out_of_bounds='clip').fit(y_prob_cali, y_cali)
        print(f"Isotonic Regression Min: {calibrator.X_min_}, Max: {calibrator.X_max_}")
        y_prob_rest = calibrator.predict(y_prob_rest)
    elif method == 'sigmoid':
        calibrator = LogisticRegression(solver='lbfgs', penalty=None, random_state=utils.SEED).fit(y_prob_cali.reshape(-1, 1), y_cali)
        assert get_model_size(calibrator) == 2
        y_prob_rest = calibrator.predict_proba(y_prob_rest.reshape(-1, 1))[:, 1]
    
    if save:
        joblib.dump(calibrator, f"models/{score}-calibrator")
        print(f"SAVE: Calibrator saved to models/{score}-calibrator")
    assert np.min(y_prob_rest) >= 0 and np.max(y_prob_rest) <= 1
    _, _, h, p = get_calibration_curve(y_rest, y_prob_rest, strategy='quantile')
    prob_true, prob_pred, _, _ = get_calibration_curve(y_rest, y_prob_rest)
    
    if plot:
        sns.set_style("ticks")
        sns.lineplot(x=np.linspace(0,1), y=np.linspace(0,1), color='black', linestyle='--', linewidth=1)
        ax = sns.lineplot(x=prob_pred, y=prob_true, color='red', linewidth=1, marker='s', label=f"{score}")
        ax.figure.set_size_inches(7, 7)
        plt.xlabel("Predicted Probability")
        plt.ylabel("True Probability")
        plt.legend()
        plt.show()
    smr = np.sum(y_test.replace({-1: 0})) / np.sum(y_prob_rest)
    print(f"{score}\nBrier score: {brier_score_loss(y_rest, y_prob_rest):.3f}\nHosmer-Lemeshow C-stat: {h:.3f}, p-value: {p:.3f}\nSMR: {smr:.3f}\n")

calibrate_scores(score='oasis_prob', y_rest=y_rest, y_prob_rest=oasis_rest_prob, y_cali=y_cali, y_prob_cali=oasis_cali_prob, plot=True, save=True)
calibrate_scores(score='sapsii_prob', y_rest=y_rest, y_prob_rest=sapsii_rest_prob, y_cali=y_cali, y_prob_cali=sapsii_cali_prob, plot=True, save=True)
calibrate_scores(score='apache_iv_prob', y_rest=y_rest, y_prob_rest=apacheiv_rest_prob, y_cali=y_cali, y_prob_cali=apacheiv_cali_prob, plot=True, save=True)
calibrate_scores(score='apache_iva_prob', y_rest=y_rest, y_prob_rest=apacheiva_rest_prob, y_cali=y_cali, y_prob_cali=apacheiva_cali_prob, plot=True, save=True)

In [ ]:
calibrate_fasterrisk(X_rest, y_rest, X_cali, y_cali, group_sparsity=40, plot=True, method='isotonic', save=True)
calibrate_fasterrisk(X_rest, y_rest, X_cali, y_cali, group_sparsity=15, plot=True, method='isotonic', save=True)
calibrate_fasterrisk(X_rest, y_rest, X_cali, y_cali, group_sparsity=10, plot=True, method='isotonic', save=True)